# Volumetric heat maps

In [1]:
samples = 1

In [2]:
import time
import random
import math
import numpy as np

In [3]:
%env QRACK_QUNITMULTI_DEVICES 1
from pyqrack import QrackSimulator, QrackCircuit
from qiskit import QuantumCircuit

env: QRACK_QUNITMULTI_DEVICES=1


In [4]:
def mcx(circ, qa_circ, c, q):
    circ.ucmtrx([c], [0, 1, 1, 0], q, 1)
    qa_circ.cx(c, q)

def mcy(circ, qa_circ, c, q):
    circ.ucmtrx([c], [0, -1j, 1j, 0], q, 1)
    qa_circ.cy(c, q)

def mcz(circ, qa_circ, c, q):
    circ.ucmtrx([c], [1, 0, 0, -1], q, 1)
    qa_circ.cz(c, q)

def macx(circ, qa_circ, c, q):
    circ.ucmtrx([c], [0, 1, 1, 0], q, 0)
    qa_circ.x(c)
    qa_circ.cx(c, q)
    qa_circ.x(c)

def macy(circ, qa_circ, c, q):
    circ.ucmtrx([c], [0, -1j, 1j, 0], q, 0)
    qa_circ.x(c)
    qa_circ.cy(c, q)
    qa_circ.x(c)

def macz(circ, qa_circ, c, q):
    circ.ucmtrx([c], [1, 0, 0, -1], q, 0)
    qa_circ.x(c)
    qa_circ.cz(c, q)
    qa_circ.x(c)

def rand_u3(circ, qa_circ, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)
    qa_circ.u(th, ph, lm, q)

    c = math.cos(th / 2)
    s = math.sin(th / 2)

    op = []
    op.append(c)
    op.append(-np.exp(1j * lm) * s)
    op.append(np.exp(1j * ph) * s)
    op.append(np.exp(1j * (ph + lm)) * c)

    circ.mtrx(op, q)

qrack_circs = []
aer_circs = []

def bench(width, depth, sdrp):
    sim = QrackSimulator(width)
    sim.set_sdrp(sdrp)
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    two_qubit_gates = mcx, mcy, mcz, macx, macy, macz
    
    colLen = math.floor(math.sqrt(width))
    while ((math.floor(width / colLen) * colLen) != width):
        colLen = colLen - 1
    rowLen = width // colLen;

    time_results = []
    fidelity_results = []

    for _ in range(samples):
        circ = QrackCircuit()
        qa_circ = QuantumCircuit(width, width)
        d_time_results = []
        d_fidelity_results = []
        
        for i in range(depth):
            # Single bit gates
            for j in range(width):
                rand_u3(circ, qa_circ, j)

            gate = gateSequence[0]
            gateSequence.pop(0)
            gateSequence.append(gate)

            for row in range(1, rowLen, 2):
                for col in range(0, colLen):
                    tempRow = row
                    tempCol = col

                    tempRow = tempRow + (1 if (gate & 2) else -1)
                    if colLen != 1:
                        tempCol = tempCol + (1 if (gate & 1) else 0)

                if (tempRow < 0) or (tempCol < 0) or (tempRow >= rowLen) or (tempCol >= colLen):
                    continue;

                b1 = row * colLen + col;
                b2 = tempRow * colLen + tempCol;

                # Two bit gates
                choice = random.choice(two_qubit_gates)
                choice(circ, qa_circ, b1, b2)

            qrack_circs.append(circ)
            aer_circs.append(qa_circ)
                
            sim.reset_all()
            start = time.time()
            circ.run(sim)
            d_time_results.append(time.time() - start)
            d_fidelity_results.append(sim.get_unitary_fidelity())

        if not time_results:
            for i in range(len(d_time_results)):
                time_results.append(d_time_results[i] / samples)
            for i in range(len(d_fidelity_results)):
                fidelity_results.append(d_fidelity_results[i] / samples)
        else:
            for i in range(len(d_time_results)):
                time_results[i] += d_time_results[i] / samples
            for i in range(len(d_fidelity_results)):
                fidelity_results[i] += d_fidelity_results[i] / samples
        
    return (time_results, fidelity_results)

In [5]:
qrack_time_results = {}
qrack_fidelity_results = {}

#for n in [36, 49, 64, 81, 100]:
for n in [36]:
    qrack_time_results[n] = []
    qrack_fidelity_results[n] = []

    sdrp = 1

    while (sdrp > 0.01):
        width_results = []
        
        # Run the benchmarks
        tmp = bench(n, n, sdrp)
        
        qrack_time_results[n].append(tmp[0])
        qrack_fidelity_results[n].append(tmp[1])
        
        sdrp -= 0.0125

print(qrack_time_results, qrack_fidelity_results)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_Gen9_HD_Graphics_NEO.ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_2070_Super.ir
{36: [[6.723403930664062e-05, 4.076957702636719e-05, 4.744529724121094e-05, 0.00019288063049316406, 0.00021219253540039062, 0.00017380714416503906, 0.0002911090850830078, 0.0001251697540283203, 0.0002090930938720703, 0.00013947486877441406, 0.0002200603485107422, 0.00015306472778320312, 0.00024700164794921875, 0.0001533031463623047, 0.00024390220642089844, 0.00022673606872558594, 0.0003228187561035156, 0.00022602081298828125, 0.00041937828063964844, 0.00023412704467773438, 0.00031304359436035156, 0.0002295970916748047, 0.0003323554992675781, 0.0003063678741455078, 0.0004904270172119141, 0.00036525726318359375, 0.00047206878662109375, 0.00031828880310058594, 0.0004038810729980469, 0.0003287792205810547, 0.00044465065002441406, 0.00038695335388183594, 0.0005104541778564453, 0.00039768218994140625, 0

In [6]:
print(aer_circs[0])

        ┌────────────────────────┐  ┌─────────────────────────┐ »
 q_0: ──┤ U(6.3694,5.5954,2.813) ├──┤ U(8.6196,9.9727,4.7412) ├─»
      ┌─┴────────────────────────┴┐┌┴─────────────────────────┴┐»
 q_1: ┤ U(4.9903,0.22542,0.86579) ├┤ U(0.36498,0.53751,9.9336) ├»
      └┬──────────────────────────┤└┬─────────────────────────┬┘»
 q_2: ─┤ U(11.726,0.14763,2.0555) ├─┤ U(6.6192,8.3351,6.7388) ├─»
       ├─────────────────────────┬┘ └┬────────────────────────┤ »
 q_3: ─┤ U(1.7195,2.3681,5.4503) ├───┤ U(11.172,7.8302,9.213) ├─»
       └┬───────────────────────┬┘  ┌┴────────────────────────┤ »
 q_4: ──┤ U(5.8207,9.671,1.066) ├───┤ U(9.5893,11.097,5.2601) ├─»
       ┌┴───────────────────────┴┐  └──────────┬───┬──────────┘ »
 q_5: ─┤ U(12.193,2.659,0.10916) ├─────────────┤ X ├────────────»
       ├─────────────────────────┴┐            └─┬─┘            »
 q_6: ─┤ U(8.0332,4.4308,0.72157) ├──────────────┼──────────────»
       ├─────────────────────────┬┘              │              »
 q_7: ─┤ U

In [ ]:
print(qrack_circs[0])

## Results

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 36
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[36]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=36.455696203)

fig.savefig('heat_map_time_36.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 49
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[49]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=49.620253165)

fig.savefig('heat_map_time_49.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 64
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[64]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=64.810126582)

fig.savefig('heat_map_time_64.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 81
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[81]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=82.025316456)

fig.savefig('heat_map_time_81.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 100
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[100]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=101.2658)

fig.savefig('heat_map_time_100.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 36
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[36]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=36.455696203)

fig.savefig('heat_map_fidelity_36.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 49
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[49]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=49.620253165)

fig.savefig('heat_map_fidelity_49.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 64
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[64]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=64.810126582)

fig.savefig('heat_map_fidelity_64.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 81
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[81]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=82.025316456)

fig.savefig('heat_map_fidelity_81.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 100
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[100]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=101.2658)

fig.savefig('heat_map_fidelity_100.png', dpi=100)

In [ ]:
import platform

print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(platform.processor())

In [ ]:
import subprocess

line_as_bytes = subprocess.check_output("nvidia-smi -L", shell=True)
line = line_as_bytes.decode("ascii")
_, line = line.split(":", 1)
line, _ = line.split("(")
print(line.strip())

In [ ]:
import cpuinfo
cpuinfo.get_cpu_info()['brand_raw']